# Research name

## Intro

### Task

### Imports

In [ ]:
# BASIC LIBS

# data
import pandas as pd
import numpy as np

import geopandas as gpd
import contextily as ctx
from pyproj import CRS

# stat significance
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
%config InlineBackend.figure_format = 'retina'

In [2]:
# JUPYTER CONFIG
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Config

## Data

### OD

In [5]:
df = pd.read_csv('./data/georgia_od_data/GDOT_2019_09.csv')

In [6]:
sample_size = 100000
df_sample = df.sample(sample_size)

In [7]:
df_sample.columns[0:100]

Index(['year', 'month', 'origin_zone_id', 'destination_zone_id',
       'origin_zone_name', 'destination_zone_name', 'origin_state',
       'destination_state', 'origin_flag', 'destination_flag', 'day_type',
       'avg_linked_trips', 'avg_unlinked_trips', 'avg_tours',
       'total_linked_trips', 'total_unlinked_trips', 'total_tours', 'hour_00',
       'hour_01', 'hour_02', 'hour_03', 'hour_04', 'hour_05', 'hour_06',
       'hour_07', 'hour_08', 'hour_09', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'distance_0_1',
       'distance_1_2.5', 'distance_2.5_5', 'distance_5_10', 'distance_10_25',
       'distance_25_50', 'distance_50_75', 'distance_75_100',
       'distance_100_150', 'distance_150_300', 'distance_gt300', 'mode_air',
       'mode_rail', 'mode_car', 'mode_bus', 'mode_walk', 'mode_bike',
       'purpose_hbw', 'purpose_hbo', 'purpose_wbo', 'purpose_obo', 

In [8]:
total_trips = 26842386

In [9]:
df_sample[['total_linked_trips', 'distance_0_1',
       'distance_1_2.5', 'distance_2.5_5', 'distance_5_10', 'distance_10_25',
       'distance_25_50', 'distance_50_75', 'distance_75_100',
       'distance_100_150', 'distance_150_300', 'distance_gt300']].sum() / total_trips * 100

total_linked_trips    99.846545
distance_0_1          18.218589
distance_1_2.5        19.252141
distance_2.5_5        18.366933
distance_5_10         19.430888
distance_10_25        16.395864
distance_25_50         4.595366
distance_50_75         0.979686
distance_75_100        0.451268
distance_100_150       0.593200
distance_150_300       0.830891
distance_gt300         0.731720
dtype: float64

In [10]:
df_sample[['total_linked_trips', 'mode_air', 'mode_rail',
       'mode_car', 'mode_bus', 'mode_walk', 'mode_bike', 'purpose_hbw',
       'purpose_hbo', 'purpose_wbo', 'purpose_obo']].sum() / total_trips * 100

total_linked_trips    99.846545
mode_air               0.546214
mode_rail              0.429876
mode_car              86.357226
mode_bus               3.193759
mode_walk              8.716472
mode_bike              0.602998
purpose_hbw           14.639291
purpose_hbo           48.240671
purpose_wbo           11.010322
purpose_obo           25.956262
dtype: float64

In [11]:
# 26842386 is sum of total_linked_trips
df_sample[['total_linked_trips', 'hour_00',
       'hour_01', 'hour_02', 'hour_03', 'hour_04', 'hour_05', 'hour_06',
       'hour_07', 'hour_08', 'hour_09', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']].sum() / total_trips * 100

total_linked_trips    99.846545
hour_00                0.718699
hour_01                0.619483
hour_02                0.580634
hour_03                0.523161
hour_04                0.744140
hour_05                1.279640
hour_06                3.557195
hour_07                5.939003
hour_08                5.283100
hour_09                5.190992
hour_10                5.874310
hour_11                6.610694
hour_12                7.302376
hour_13                7.273679
hour_14                7.392297
hour_15                7.767435
hour_16                7.240929
hour_17                7.650866
hour_18                6.948730
hour_19                4.687773
hour_20                2.469076
hour_21                1.829629
hour_22                1.375168
hour_23                0.987535
dtype: float64

In [12]:
# how many destination may one origin have
(
df
.groupby(["origin_zone_id"])
.agg({
    "origin_zone_id":"count",
})
.rename(columns={"origin_zone_id": "count"})
.reset_index()
.sort_values(['count'], ascending=False)
[:5]
)

,origin_zone_id,count
1383,130639800001,7178
3450,131210035001,5693
7501,13820_AL,5349
7531,16740_NC,4514
3677,131210092032,4387


### Block Groups shp

In [13]:
gdf_zones = gpd.read_file(r"./data/block_groups_shapefiles_2019/tl_2019_13_bg.shp")

In [14]:
# gdf_zones["GEOID"] = gdf_zones["GEOID"].str[:-1]

In [15]:
# gdf_zones_sample = gdf_zones.sample(3000)
gdf_zones_sample = gdf_zones

In [16]:
CRS_mercator = CRS.from_string("epsg:3857")
gdf_zones_plot = gdf_zones_sample.to_crs(CRS_mercator)
gdf_zones_plot.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
gdf_zones_plot.explore()

### Population By Tract

In [202]:
df_tr_pop = pd.read_csv('./data/population_by_tract_2021.csv')

#### Tract as a proxy to block group

In [203]:
df_tr_pop = df_tr_pop.rename({"Geographic ID for geographic unit":"GEOID", "# Total population, 2021":"POP"}, axis=1)
gdf_zones["GEOID"] = gdf_zones["GEOID"].str[:-1]
df_join = (
    gdf_zones
    [["GEOID", "BLKGRPCE"]]
    .merge(df_tr_pop[["GEOID", "POP"]].astype(str), on = "GEOID", how = "left")
)


<span style="color:yellow">*Tract of 40% of block groups is missing!!!*</span>

In [204]:
df_join.POP.isna().value_counts()

POP
False    3155
True     2378
Name: count, dtype: int64

In [206]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_join):
    # Rename column 'GEOID' to 'tract_id'
    df_join = df_join.rename(columns={'GEOID': 'tract_id'})
    # Rename column 'BLKGRPCE' to 'block_number'
    df_join = df_join.rename(columns={'BLKGRPCE': 'block_number'})
    # Rename column 'block_number' to 'block'
    df_join = df_join.rename(columns={'block_number': 'block'})
    # Rename column 'POP' to 'tract_population'
    df_join = df_join.rename(columns={'POP': 'tract_population'})
    # Performed 1 aggregation grouped on columns: 'tract_id', 'tract_population'
    df_join = df_join.groupby(['tract_id', 'tract_population']).agg(block_count=('block', 'count')).reset_index()
    # Rename column 'block_count' to 'number_of_blocks'
    df_join = df_join.rename(columns={'block_count': 'number_of_blocks'})
    return df_join

df_join_clean = clean_data(df_join.copy())

def clean_data(df_join_clean):
    # Change column type to int64 for column: 'number_of_blocks'
    df_join_clean = df_join_clean.astype({'number_of_blocks': 'int64'})
    # Change column type to int64 for column: 'tract_population'
    df_join_clean = df_join_clean.astype({'tract_population': 'int64'})
    # Derive column 'derivedCol' from columns: 'tract_population', 'number_of_blocks'
    df_join_clean.insert(3, 'derivedCol', df_join_clean.apply(lambda row : row.tract_population / row.number_of_blocks, axis=1))
    return df_join_clean

df_join_clean_1 = clean_data(df_join_clean.copy())
df_join_clean_1.head()

,tract_id,tract_population,number_of_blocks,derivedCol
0,13001950100,3421,2,1710.500000
1,13001950400,1852,2,926.000000
2,13001950500,3723,3,1241.000000
3,13003960100,2214,2,1107.000000
4,13003960200,4604,3,1534.666667


## Main

### 

## Other

#### Viz of Blocks with Trip amounts

In [471]:
gdf_zones = gpd.read_file(r"./data/block_groups_shapefiles_2019/tl_2019_13_bg.shp")

In [473]:
def clean_data(df_sample):
    # Performed 2 aggregations grouped on column: 'origin_zone_id'
    df_sample = df_sample.groupby(['origin_zone_id', 'origin_state']).agg(destination_zone_id_count=('destination_zone_id', 'count'), total_linked_trips_sum=('total_linked_trips', 'sum')).reset_index()
    return df_sample

df_origins = clean_data(df_sample.copy())
df_origins

,origin_zone_id,origin_state,destination_zone_id_count,total_linked_trips_sum
0,1005,AL,12,916
1,1017,AL,31,5163
2,1019,AL,16,3313
3,1029,AL,19,2558
4,1049,AL,14,1273
...,...,...,...,...
7876,RWI1_WI,WI,7,40
7877,RWI3_WI,WI,3,123
7878,RWV1_WV,WV,3,14
7879,RWV2_WV,WV,4,18


In [474]:
# !!!
df_origins["origin_zone_id"] = df_origins["origin_zone_id"].astype(str).str[:-1]
gdf_zones["GEOID"] = gdf_zones["GEOID"].astype(str).str[:-1]

In [464]:
gdf_origins = (
    df_origins.query("origin_state == 'GA'")
    .merge(gdf_zones[["GEOID", "geometry"]], how = "left", left_on="origin_zone_id", right_on="GEOID")
)

print(gdf_origins.isna().value_counts())
gdf_origins = gdf_origins.dropna()

origin_zone_id  origin_state  destination_zone_id_count  total_linked_trips_sum  GEOID  geometry
False           False         False                      False                   False  False       10089
                                                                                 True   True         3769
Name: count, dtype: int64


In [466]:
gdf_origins

,origin_zone_id,origin_state,destination_zone_id_count,total_linked_trips_sum,GEOID,geometry
0,13003960300,GA,2,170,13003960300,"POLYGON ((-82.81773 31.35824, -82.81744 31.358..."
1,13005970100,GA,5,3126,13005970100,"POLYGON ((-82.53976 31.64450, -82.53951 31.644..."
2,13005970100,GA,5,3126,13005970100,"POLYGON ((-82.44913 31.54280, -82.44863 31.543..."
3,13005970100,GA,5,3126,13005970100,"POLYGON ((-82.44642 31.53904, -82.44604 31.539..."
4,13005970100,GA,3,52,13005970100,"POLYGON ((-82.53976 31.64450, -82.53951 31.644..."
...,...,...,...,...,...,...
13853,13321950500,GA,11,6208,13321950500,"POLYGON ((-83.99746 31.47632, -83.99745 31.477..."
13854,13321950600,GA,8,635,13321950600,"POLYGON ((-83.94037 31.37549, -83.94036 31.376..."
13855,13321950600,GA,8,635,13321950600,"POLYGON ((-83.99941 31.33513, -83.99941 31.335..."
13856,13321950600,GA,7,1046,13321950600,"POLYGON ((-83.94037 31.37549, -83.94036 31.376..."


#### Other

In [ ]:
gdf_t1 = gpd.read_file(r"./data/illinois_tr_shp/tl_2019_17_tract.shp")

CRS_mercator = CRS.from_string("epsg:3857")
gdf_t1 = gdf_t1.to_crs(CRS_mercator)

gdf_t1.explore()

### 

same origin and destinations do exist. for some reason with TWO different rows (two diff data on amount of trips from loc to itself)

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Filter rows based on column: 'origin_zone_id'
    df = df[df['origin_zone_id'].apply(str) == "130570910013"]
    # Filter rows based on column: 'destination_zone_id'
    df = df[df['destination_zone_id'] == "130570910013"]
    return df

df_clean = clean_data(df.copy())
df_clean.head()

od data is two-directional

### Gurobi tests

In [1]:
import gurobipy as gp

In [2]:
!gurobi_cl

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Using license file /Library/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-10-20

Usage: gurobi_cl [--command]* [param=value]* filename
Type 'gurobi_cl --help' for more information.


In [ ]:
# !pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 10.2 MB/s eta 0:00:0000:0100:01


In [3]:
!echo $CONDA_DEFAULT_ENV


spatial_analysis_course


solution 3

In [ ]:
# Fixed capacity of each charging spot
cap_spot = 150

# Maximum number of charging spots per site
max_chargers = 5

# Total budget limit for the number of chargers across all sites
chargers_budget_limit = 10

# Sample data
A = ["Area1", "Area2", "Area3", "Area4"]  # All areas where EV drivers are based
P = ["Area1", "Area3"]  # Areas with potential sites for EV charging stations (subset of A)

# Number of trips from each site to each area
tr = {
    ("Area1", "Area1"): 100,
    ("Area1", "Area2"): 200,
    ("Area1", "Area3"): 50,
    ("Area3", "Area1"): 150,
    ("Area3", "Area2"): 300,
    ("Area3", "Area4"): 100,
}

# Capacity of each area, representing the demand from EV owners
c = {
    "Area1": 500,
    "Area2": 700,
    "Area3": 300,
    "Area4": 400
}

In [ ]:
import random

# Fixed capacity of each charging spot
cap_spot = 150

# Maximum number of charging spots per site
max_chargers = 5

# Total budget limit for the number of chargers across all sites
chargers_budget_limit = 10

N_AREAS = 20
N_POTENTIAL_SITES = 5

# Define the areas and potential station sites
A = [f"Area{i}" for i in range(1, N_AREAS)]  # 20 areas

# Randomly select 10-20% of areas to be potential station sites
# num_potential_sites = random.randint(int(0.1 * len(A)), int(0.2 * len(A)))  # 10-20% of areas
P = random.sample(A, N_POTENTIAL_SITES)  # Randomly choose potential sites from areas

# Hardcoded number of trips from each potential station site to each area
# Note: Only a subset of area pairs have trips to reflect realistic connections
tr = {
    ("Area2", "Area1"): 120, ("Area2", "Area3"): 250, ("Area2", "Area6"): 300, ("Area2", "Area10"): 150, ("Area2", "Area15"): 400,
    ("Area5", "Area2"): 180, ("Area5", "Area7"): 220, ("Area5", "Area11"): 100, ("Area5", "Area14"): 340, ("Area5", "Area18"): 290,
    ("Area8", "Area4"): 90, ("Area8", "Area5"): 160, ("Area8", "Area9"): 450, ("Area8", "Area13"): 110, ("Area8", "Area17"): 230,
    ("Area12", "Area6"): 130, ("Area12", "Area10"): 280, ("Area12", "Area11"): 160, ("Area12", "Area15"): 210, ("Area12", "Area19"): 320,
    ("Area16", "Area8"): 70, ("Area16", "Area12"): 190, ("Area16", "Area13"): 300, ("Area16", "Area16"): 270, ("Area16", "Area20"): 260
}

# Hardcoded capacities for each area (demand capacity of each area)
c = {
    "Area1": 500, "Area2": 600, "Area3": 750, "Area4": 480, "Area5": 650,
    "Area6": 700, "Area7": 520, "Area8": 560, "Area9": 680, "Area10": 460,
    "Area11": 640, "Area12": 590, "Area13": 620, "Area14": 610, "Area15": 730,
    "Area16": 540, "Area17": 510, "Area18": 770, "Area19": 550, "Area20": 580
}



random data:

In [14]:
import random

N_AREAS = 2000
N_POTENTIAL_SITES = 500
chargers_budget_limit = 50 # Total budget limit for the number of chargers across all sites
max_chargers_per_site = 25 # Maximum number of charging spots per site


# Define the areas and potential station sites
A = [f"Area{i}" for i in range(1, N_AREAS)]  # 20 areas

# Randomly select 10-20% of areas to be potential station sites
# num_potential_sites = random.randint(int(0.1 * len(A)), int(0.2 * len(A)))  # 10-20% of areas
P = random.sample(A, N_POTENTIAL_SITES)  # Randomly choose potential sites from areas

# Randomly generate trips from each potential station site to each area
# Note: Each station site serves a random subset of areas, with trips varying between 50 and 500
tr = {}
for i in P:
    for j in A:
        if random.random() < 0.6:  # 60% chance that a station site serves a given area
            tr[(i, j)] = random.randint(50, 500)

# Random capacities for each area (between 400 and 1000)
c = {j: random.randint(400, 1000) for j in A}

# Fixed capacity of each charging spot
cap_spot = 150



real city like data:

In [ ]:
import pandas as pd

import random

# Parameters
N_AREAS = 10
N_POTENTIAL_SITES = 5

# Define areas
A = [f"Area{i}" for i in range(1, N_AREAS + 1)]  # 20 areas

# Randomly select potential sites from areas
P = random.sample(A, N_POTENTIAL_SITES)  # 10 areas as potential sites

# Define hubs and residential areas
# Assume first half of P are hubs and second half are residential areas
hubs = A[:N_AREAS // 2]
residential_areas = A[N_AREAS // 2:]

# Generate trips based on area types
tr = {}
for i in P:
    for j in A:
        if i in hubs:
            # Hubs receive higher inbound trips from residential areas
            if j in residential_areas:
                tr[(i, j)] = random.randint(800, 1200)
            else:
                tr[(i, j)] = random.randint(700, 1100)
        elif i in residential_areas:
            # Residential areas have higher outbound trips to hubs
            if j in hubs:
                tr[(i, j)] = random.randint(800, 1200)
            else:
                tr[(i, j)] = random.randint(150, 300)

# Random capacities for each area (higher for residential areas)
c = {}
for j in A:
    if j in residential_areas:
        c[j] = random.randint(800, 1000)  # Higher population for residential areas
    else:
        c[j] = random.randint(100, 200)  # Lower population for hubs



# Convert the trips dictionary `tr` to a DataFrame
tr_matrix = pd.DataFrame(0, index=sorted(P), columns=A)  # Initialize with zeros

# Populate the DataFrame with trip values
for (i, j), value in tr.items():
    tr_matrix.loc[i, j] = value

# Display trips matrix
print("\nTrips Matrix (Potential Station Sites as rows, Areas as columns):")
print(tr_matrix)

# Convert capacities dictionary `c` to a DataFrame
c_df = pd.DataFrame.from_dict(c, orient='index', columns=['Capacity'])

# Display capacities matrix
print("\nArea Capacities (c):")
print(c_df.T)  # Transpose for a row-style display

# Constants for the model
print("\nConstants:")
print("cap_spot:", CAP_SPOT)
print("max_chargers:", MAX_CHARGERS)
print("chargers_budget_limit:", CHARGERS_BUDGET_LIMIT)



Trips Matrix (Potential Station Sites as rows, Areas as columns):
        Area1  Area2  Area3  Area4  Area5  Area6  Area7  Area8  Area9  Area10
Area1     995    738   1028    756    908    875   1196   1012    863     873
Area10    915   1085    890   1103    851    179    190    226    235     188
Area4     867    893    845   1042    976   1163   1033   1018    892     813
Area5     972    817    723    869    757    903    989   1097    889     878
Area6     902    970    803    811    831    152    174    180    168     207

Area Capacities (c):
          Area1  Area2  Area3  Area4  Area5  Area6  Area7  Area8  Area9  \
Capacity    128    112    146    135    109    879    990    910    859   

          Area10  
Capacity     997  

Constants:
cap_spot: 150
max_chargers: 5
chargers_budget_limit: 20


In [56]:
from gurobipy import Model, GRB, quicksum


CHARGERS_BUDGET_LIMIT = 15
CAP_SPOT = 200  # Each charging spot serves up to 150 trips
MAX_CHARGERS = 5  # Maximum number of chargers per site



# Create a new model
model = Model("EV Charging Station Placement with Charger Capacities")

# Decision variables
build = model.addVars(P, vtype=GRB.INTEGER, lb=0, ub=MAX_CHARGERS, name="build")  # Number of charging spots at each site
served = model.addVars(P, A, vtype=GRB.CONTINUOUS, lb=0, name="served")  # Trips served from site i to area j
z = model.addVars(A, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="z")  # Saturation level for each area

# Auxiliary variable for raw saturation calculation
saturation_raw = model.addVars(A, vtype=GRB.CONTINUOUS, lb=0, name="saturation_raw")

# Objective: Maximize the total effective coverage across all areas
model.setObjective(quicksum(z[j] * c[j] for j in A), GRB.MAXIMIZE)


# Constraints
# Capacity constraint: Total trips served from site i to all areas cannot exceed the capacity from installed chargers
for i in P:
    model.addConstr(quicksum(served[i, j] for j in A) <= build[i] * CAP_SPOT, name=f"capacity_constraint_{i}")



# Trip limit constraint: Trips served from site i to area j cannot exceed the actual trips
for i, j in tr:
    model.addConstr(served[i, j] <= tr[i, j], name=f"trip_limit_{i}_{j}")



# Raw saturation calculation for each area
for j in A:
    model.addConstr(saturation_raw[j] == quicksum(served[i, j] for i in P if (i, j) in tr) / c[j], name=f"saturation_raw_{j}")



# Saturation calculation: z[j] is the minimum of 1 and saturation_raw[j]
for j in A:
    model.addGenConstrMin(z[j], [saturation_raw[j], 1], name=f"saturation_{j}")

# New constraint: Limit the total number of chargers built across all sites to chargers_budget_limit
model.addConstr(quicksum(build[i] for i in P) <= CHARGERS_BUDGET_LIMIT, name="chargers_budget_limit")

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in P:
        if build[i].x > 0.5:
            print(f"Build {int(build[i].x)} charging spots at site {i}")
    for j in A:
        a = 1
        print(f"Effective coverage for area {j}: {z[j].x * 100:.2f}% of capacity")

else:
    print("No optimal solution found.")



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 66 rows, 75 columns and 170 nonzeros
Model fingerprint: 0x49558e9d
Model has 10 general constraints
Variable types: 70 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+02]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 5e+00]
  RHS range        [2e+01, 1e+03]
  GenCon const rng [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 50 rows and 10 columns
Presolve time: 0.00s
Presolved: 16 rows, 65 columns, 120 nonzeros
Variable types: 60 continuous, 5 integer (0 binary)

Root relaxation: objective 3.000000e+03, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [49]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in P:
        if build[i].x > 0.5:
            print(f"Build {int(build[i].x)} charging spots at site {i}")
            for j in A:
                if served[i,j].x > 0:
                    print(f"  Serve {served[i,j].x} trips to {j}")
    for j in A:
        print(f"Effective coverage for area {j}: {z[j].x * 100:.2f}% of capacity")

Optimal solution found:
Build 5 charging spots at site Area5
  Serve 135.0 trips to Area4
  Serve 865.0 trips to Area8
Build 4 charging spots at site Area4
  Serve 112.0 trips to Area2
  Serve 688.0 trips to Area10
Build 5 charging spots at site Area6
  Serve 118.0 trips to Area1
  Serve 146.0 trips to Area3
  Serve 109.0 trips to Area5
  Serve 152.0 trips to Area6
  Serve 174.0 trips to Area7
  Serve 168.0 trips to Area9
  Serve 133.0 trips to Area10
Build 1 charging spots at site Area10
  Serve 10.0 trips to Area1
  Serve 190.0 trips to Area7
Effective coverage for area Area1: 100.00% of capacity
Effective coverage for area Area2: 100.00% of capacity
Effective coverage for area Area3: 100.00% of capacity
Effective coverage for area Area4: 100.00% of capacity
Effective coverage for area Area5: 100.00% of capacity
Effective coverage for area Area6: 17.29% of capacity
Effective coverage for area Area7: 36.77% of capacity
Effective coverage for area Area8: 95.05% of capacity
Effective co

In [51]:
a = [1, 2]
a[3]

IndexError: list index out of range